<b> First, check to make sure MegaDetector is available.
<br> CameraTraps, ai4eutils, and yolo5 must be included in the PYTHONPATH

In [15]:
import pandas as pd
import sys
sys.path

['/home/kyra/animl-py/src/animl',
 '/home/kyra/animl-py/src/animl',
 '/home/kyra/CameraTraps',
 '/home/kyra/ai4eutils',
 '/home/kyra/yolov5',
 '/home/kyra/anaconda3/envs/cameratraps-detector/lib/python38.zip',
 '/home/kyra/anaconda3/envs/cameratraps-detector/lib/python3.8',
 '/home/kyra/anaconda3/envs/cameratraps-detector/lib/python3.8/lib-dynload',
 '',
 '/home/kyra/.local/lib/python3.8/site-packages',
 '/home/kyra/anaconda3/envs/cameratraps-detector/lib/python3.8/site-packages']

<b> Find file paths for the image directory of interest, both model files, and the classifier class list.

In [3]:
imagedir = "/home/kyra/animl-py/examples/Peru/S90/SDZG 2"
model_file = "/mnt/machinelearning/megaDetector/md_v5b.0.0.pt"
class_model = "/mnt/machinelearning/Models/Southwest/2022/EfficientNetB5_456_Unfrozen_05_0.26_0.92.h5"
classes = "/mnt/machinelearning/Models/Southwest/2022/classes.txt"

<b> Create a working directory and build the file manifest from the image directory.

In [4]:
import fileManagement
workingdir = fileManagement.WorkingDirectory(imagedir)

In [5]:
files = fileManagement.build_file_manifest(imagedir, out_file = workingdir.filemanifest)

Output file already exists and was last modified 2023-05-31 11:07:07.954538,                  would you like to load it? y/n: y


<b> If there are videos, save out individual frames to pass through models. 

In [6]:
import videoProcessing
allframes = videoProcessing.images_from_videos(files, out_dir=workingdir.vidfdir, 
                                               out_file=workingdir.imageframes, parallel = True, frames=2)

Output file already exists and was last modified 2023-06-06 15:39:28.040917,                  would you like to load it? y/n: y


<b> Run all images and video frames through MegaDetector

In [7]:
import detectMD
mdresults = detectMD.detect_MD_batch(model_file, allframes["Frame"], checkpoint_path=None,
                                checkpoint_frequency=-1, results=None, n_cores=1, quiet=True)

PyTorch reports 1 available CUDA devices
GPU available: True
Using PyTorch version 1.10.1


Fusing layers... 
Model summary: 574 layers, 139990096 parameters, 0 gradients


Sending model to GPU
Loaded model in 7.64 seconds
Loaded model in 7.64 seconds


100%|███████████████████████████████████████████| 85/85 [00:07<00:00, 11.05it/s]


<b> Convert MD JSON to pandas dataframe, merge with manifest.

In [8]:
import parseResults
mdres = parseResults.parseMD(mdresults, manifest=allframes, out_file = workingdir.mdresults)

Output file already exists and was last modified 2023-06-12 15:40:13.456840,                  would you like to load it? y/n: y


<b> Extract animal detections from the rest.

In [9]:
import splitData
animals = splitData.getAnimals(mdres)
empty = splitData.getEmpty(mdres)

<b> Use the classifier model to predict the species of animal detections.

In [10]:
import predictSpecies
predresults = predictSpecies.predictSpecies(animals, class_model, batch = 4)

2023-06-13 16:50:19.777936: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-13 16:50:19.979417: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-06-13 16:50:20.558152: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-06-13 16:50:20.558219: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

37


2023-06-13 16:50:31.238863: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8500


 1/37 [..............................] - ETA: 2:12

2023-06-13 16:50:31.693087: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-06-13 16:50:31.740640: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


37/37 [==============================] - 8s 112ms/step


In [11]:
import parseResults
animals = parseResults.applyPredictions(animals,predresults,classes, out_file = workingdir.predictions)

In [12]:
animals.columns

Index(['FilePath', 'FileName', 'FileModifyDate', 'Frame', 'file',
       'max_detection_conf', 'category', 'conf', 'bbox1', 'bbox2', 'bbox3',
       'bbox4', 'prediction', 'confidence'],
      dtype='object')

In [13]:
empty.columns

Index(['FilePath', 'FileName', 'FileModifyDate', 'Frame', 'file',
       'max_detection_conf', 'category', 'conf', 'bbox1', 'bbox2', 'bbox3',
       'bbox4', 'prediction', 'confidence'],
      dtype='object')

In [16]:
manifest = pd.concat([animals,empty])

In [17]:
manifest

,FilePath,FileName,FileModifyDate,Frame,file,...,bbox2,bbox3,bbox4,prediction,confidence
0,/home/kyra/animl-py/examples/Peru/S90/SDZG 2/E...,EK000026.JPG,2022-08-25 11:50:17,/home/kyra/animl-py/examples/Peru/S90/SDZG 2/E...,/home/kyra/animl-py/examples/Peru/S90/SDZG 2/E...,...,0.83980,0.02000,0.11190,human,0.043124
1,/home/kyra/animl-py/examples/Peru/S90/SDZG 2/E...,EK000026.JPG,2022-08-25 11:50:17,/home/kyra/animl-py/examples/Peru/S90/SDZG 2/E...,/home/kyra/animl-py/examples/Peru/S90/SDZG 2/E...,...,0.75240,0.13140,0.08741,bobcat,0.119125
2,/home/kyra/animl-py/examples/Peru/S90/SDZG 2/E...,EK000022.JPG,2022-08-25 11:50:17,/home/kyra/animl-py/examples/Peru/S90/SDZG 2/E...,/home/kyra/animl-py/examples/Peru/S90/SDZG 2/E...,...,0.45380,0.04197,0.03227,dog,0.011405
3,/home/kyra/animl-py/examples/Peru/S90/SDZG 2/E...,EK000036.JPG,2022-08-25 11:50:17,/home/kyra/animl-py/examples/Peru/S90/SDZG 2/E...,/home/kyra/animl-py/examples/Peru/S90/SDZG 2/E...,...,0.59470,0.14490,0.11390,cougar,0.004797
4,/home/kyra/animl-py/examples/Peru/S90/SDZG 2/E...,EK000036.JPG,2022-08-25 11:50:17,/home/kyra/animl-py/examples/Peru/S90/SDZG 2/E...,/home/kyra/animl-py/examples/Peru/S90/SDZG 2/E...,...,0.56200,0.04626,0.05147,empty,0.491225
...,...,...,...,...,...,...,...,...,...,...,...
10,/home/kyra/animl-py/examples/Peru/S90/SDZG 2/E...,EK000011.AVI,2022-08-25 11:50:17,/home/kyra/animl-py/examples/Peru/S90/SDZG 2/A...,/home/kyra/animl-py/examples/Peru/S90/SDZG 2/A...,...,0.02000,0.03281,0.28330,human,0.907000
11,/home/kyra/animl-py/examples/Peru/S90/SDZG 2/E...,EK000011.AVI,2022-08-25 11:50:17,/home/kyra/animl-py/examples/Peru/S90/SDZG 2/A...,/home/kyra/animl-py/examples/Peru/S90/SDZG 2/A...,...,0.02000,0.03750,0.17080,human,0.907000
12,/home/kyra/animl-py/examples/Peru/S90/SDZG 2/E...,EK000011.AVI,2022-08-25 11:50:17,/home/kyra/animl-py/examples/Peru/S90/SDZG 2/A...,/home/kyra/animl-py/examples/Peru/S90/SDZG 2/A...,...,0.02777,0.12730,0.26520,human,0.907000
13,/home/kyra/animl-py/examples/Peru/S90/SDZG 2/E...,EK000011.AVI,2022-08-25 11:50:17,/home/kyra/animl-py/examples/Peru/S90/SDZG 2/A...,/home/kyra/animl-py/examples/Peru/S90/SDZG 2/A...,...,0.02000,0.10390,0.31660,human,0.907000
